In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from IPython.display import display, clear_output
import pandas as pd
import time
import json

from itertools import product
from collections import namedtuple
from collections import OrderedDict

import os 

#NameFile
class NameFile():
    @staticmethod#static? iets met dat je m kan callen using the class itself. don't need an instance of the class, to call the method. i guess dat je m dus niet eerst hoeft te initieren alszijnde type x? 
    def nameit(params): #ga er dus vanuit dat je alleen keys and values hebt, dus alleen 0 en 1 voor die ene index. wellicht gaat dit dus ooit mis, makkelijk te herstellen

        items_hier = list(params.items()) #keys+values
        num_k=0 #amount of keys present
        comment = '' #the string for the filename
        
        #make the comment by looping over keys and values
        for k in params.keys():
            comment += f'{items_hier[num_k][0]}=' #add the key
            for v in items_hier[num_k][1]:
                comment +=(f'{v}_') #add the values
            num_k+=1 #for indexing next loop to get next keys/values in list
        comment+='results'
        return comment

#RunBuilder
class RunBuilder():
    @staticmethod#static? iets met dat je m kan callen using the class itself. don't need an instance of the class, to call the method. i guess dat je m dus niet eerst hoeft te initieren alszijnde type x? 
    def get_runs(params):
        
        Run = namedtuple('Run',params.keys())#die ordereddicttionary heeft keys and values. dit heb je wel eens eerder gezien I guess.
                                             #blijkbaar maakt ie een mooie string als je die keys zo oproept. test dat even.
        
        runs = []
        for v in product(*params.values()): #dit doet dus iets dat ie per value combo nieuwe iteratie doet
            runs.append(Run(*v))
            
        return runs

#RunManager
class RunManager():
    def __init__(self):
        
        self.epoch_count = 0
        self.epoch_loss = 0
        self.epoch_num_correct = 0
        self.epoch_start_time = None
        
        self.run_params = None
        self.run_count = 0
        self.run_data = []
        self.run_start_time = None
        
        self.network = None
        self.loader = None
        self.tb = None

    def begin_run(self,run,network,loader): #die self is dus gewoon de variabel naam links van de streep
        #start time for a run, parameters run added, run_count+1 (stays same for all epochs)
        #network copied, loader copied, name given in tb. 
        self.run_start_time = time.time()
        
        self.run_params = run
        self.run_count += 1
        self.network = network
        self.loader = loader
        self.tb = SummaryWriter(comment=f'-{run}')
        
        images,labels = next(iter(self.loader)) #misschien wel gewoon plaatjes inladen voor foto'tje in tensorboard
        grid = torchvision.utils.make_grid(images)
        
        self.tb.add_image('images',grid)
        self.tb.add_graph(
            self.network
            ,images.to(getattr(run,'device','cpu')))
        
    def end_run(self):
        self.tb.close()
        self.epoch_count = 0

    def begin_epoch(self):
        self.epoch_start_time = time.time()
        
        self.epoch_count += 1
        self.epoch_loss = 0
        self.epoch_num_correct = 0
        
    def end_epoch(self):
        
        epoch_duration = time.time()-self.epoch_start_time
        run_duration = time.time()-self.run_start_time
        
        loss = self.epoch_loss/len(self.loader.dataset)
        accuracy = self.epoch_num_correct/len(self.loader.dataset)
        
        self.tb.add_scalar('Loss',loss,self.epoch_count)
        self.tb.add_scalar('Accuracy',accuracy,self.epoch_count)
        
        for name, param in self.network.named_parameters():
            self.tb.add_histogram(name,param,self.epoch_count)
            self.tb.add_histogram(f'{name}.grad',param.grad,self.epoch_count)
            
        results = OrderedDict()
        results["run"]=self.run_count
        results["epoch"]=self.epoch_count
        results["loss"]=loss
        results["accuracy"]=accuracy
        results["epoch duration"]=epoch_duration
        results["run duration"]=run_duration
                            
        for k,v in self.run_params._asdict().items():  #deze komen uit run, je batch_size & lr
            results[k] = v #geloof dat je hier dus voor elke run met andere batch size etc. maar 1 lr en batchsize toevoegt, vandaar dat dit niet in de loop zit
        self.run_data.append(results) #1 batch_size en lr bij de results bij, en vervolgens voeg je al je results toe aan wat je metadata i guess
        df = pd.DataFrame.from_dict(self.run_data,orient='columns') #dit zorgt dat het in een leuk tabelletje staat
                                
        clear_output(wait=True)
        display(df)
                                
    def track_loss(self,loss):
            self.epoch_loss += loss.item()* self.loader.batch_size
                                    
    def track_num_correct(self,preds,labels):
            self.epoch_num_correct += self._get_num_correct(preds,labels)
        
    @torch.no_grad()
    def _get_num_correct(self,preds,labels):
        return preds.argmax(dim=1).eq(labels).sum().item()
    
    def save(self,fileName,ResDir,file_num):
        
        os.mkdir(f'{ResDir}\{file_num}')
        
        pd.DataFrame.from_dict(
        self.run_data
        ,orient = 'columns'
        ).to_csv(f'{ResDir}\{file_num}\{fileName}.csv')
        
        with open(f'{ResDir}\{file_num}\{fileName}.json','w',encoding='utf-8') as f:
            json.dump(self.run_data, f, ensure_ascii=False, indent=4)

#Network
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)

        self.fc1 = nn.Linear(in_features=12*4*4, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)

    def forward(self, t):
        # (1) input layer
        t = t

        # (2) hidden conv layer
        t = self.conv1(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        # (3) hidden conv layer
        t = self.conv2(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        # (4) hidden linear layer
        t = t.reshape(-1, 12*4*4)
        t = self.fc1(t)
        t = F.relu(t)

        # (5) hidden linear layer
        t = self.fc2(t)
        t = F.relu(t)

        # (6) output layer
        t = self.out(t)
        #t = F.softmax(t,dim=1)

        return t

In [4]:
#computations
train_set = torchvision.datasets.FashionMNIST(
    root='./data/FashionMNIST', train=True, download=True, transform=transforms.Compose([transforms.ToTensor()
                                                                                         ])
)

params = OrderedDict(
    lr = [.01, .001]
    ,batch_size = [1000,10000]
)
ResDir = 'runs_results'
file_num = '0408_1538'
m=RunManager()


In [ ]:
for run in RunBuilder.get_runs(params): #[Run(lr=..,batch_size=..),Run(lr=..,batch_size=..)]
    
    network = Network() #create network
    loader = DataLoader(train_set,batch_size=run.batch_size) #load this run's data according to batch_size and lr
    optimiser = optim.Adam(network.parameters(),lr=run.lr)
    
    m.begin_run(run,network,loader) #start met tellen van een aantal zaken
    for epoch in range(5):
        m.begin_epoch() #start met tellen voor epoch specifieke zaken
        for batch in loader:
            
            images,labels=batch #laad specifieke batch
            preds = network(images) #pass batch
            loss = F.cross_entropy(preds,labels) # calculate loss
            optimiser.zero_grad() # Zero gradients
            loss.backward() # calculate gradients
            optimiser.step() # update weights
            
            m.track_loss(loss) #epoch loss is updated. each epoch at beginning it is reset to 0.
            m.track_num_correct(preds,labels) #same for num_correct
        m.end_epoch() #each epoch's accuracy and loss are written to tb. no num_correct, it's there in accuracy
    m.end_run()


In [ ]:
m.save(NameFile.nameit(params),ResDir,file_num) #mooie naam voor de results file met alle parameter values erin

In [ ]:
t = torch.ones(1,1,28,28)
network = Network()


# Moving to GPU

In [ ]:
t = t.cuda()
network = network.cuda()


In [ ]:
gpu_pred = network(t)
gpu_pred.device

# Moving to CPU

In [ ]:
t = t.cpu()
network = network.cpu()

In [ ]:
cpu_pred = network(t)
cpu_pred.device

In [ ]:
t1 = torch.tensor([
    [1,2],
    [3,4]
])

t2 = torch.tensor([
    [5,6],
    [7,8]
])

In [ ]:
t1.device, t2.device

In [ ]:
t1 = t1.to('cuda')
t1.device

In [ ]:
t2 = t2.cpu()
t2.device

In [ ]:
try: t1+t2
except Exception as e: print(e)

In [ ]:
try: t2+t1
except Exception as e: print(e)

In [ ]:
t2 = t2.to('cuda')

In [ ]:
t1+t2

In [ ]:
network = network.to('cuda')

In [ ]:
sample = torch.ones(1,1,28,28)
sample.shape

In [ ]:
try: network(sample)
except Exception as e: print(e)

In [ ]:
try:
    pred = network(sample.to('cuda'))
    print(pred)
except Exception as e:
    print(e)

In [ ]:
torch.cuda.is_available()

In [9]:
params = OrderedDict(
    lr = [.01]
    ,batch_size = [1000, 10000, 20000]
    ,num_workers = [0, 1]
    , device = ['cuda','cpu']
)
m=RunManager()
for run in RunBuilder.get_runs(params):
    
    device = torch.device(run.device)
    network = Network().to(device)
    loader = DataLoader(train_set,batch_size=run.batch_size,num_workers = run.num_workers)
    optimiser = optim.Adam(network.parameters(),lr=run.lr)
    
    m.begin_run(run,network,loader)
    for epoch in range(1):
        m.begin_epoch()
        for batch in loader:
            
            images = batch[0].to(device)
            labels = batch[1].to(device)
            preds = network(images) #pass batch
            loss = F.cross_entropy(preds,labels)
            optimiser.zero_grad()
            loss.backward()
            optimiser.step()
            
            m.track_loss(loss)
            m.track_num_correct(preds,labels)
        m.end_epoch()
    m.end_run()

,run,epoch,loss,accuracy,epoch duration,run duration,lr,batch_size,num_workers,device
0,1,1,1.046105,0.598533,6.778939,7.394323,0.01,1000,0,cuda
1,2,1,1.059174,0.583317,26.202152,27.456803,0.01,1000,0,cpu
2,3,1,1.023789,0.602033,9.261346,12.786363,0.01,1000,1,cuda
3,4,1,0.974529,0.631817,19.735613,22.791895,0.01,1000,1,cpu
4,5,1,2.148473,0.185933,11.915739,17.451818,0.01,10000,0,cuda
5,6,1,2.087540,0.271700,17.762079,27.343355,0.01,10000,0,cpu
6,7,1,2.148680,0.180700,7.740357,14.333302,0.01,10000,1,cuda
7,8,1,2.137560,0.199417,14.550092,24.784825,0.01,10000,1,cpu
8,9,1,2.289859,0.127833,10.266546,17.618935,0.01,20000,0,cuda
9,10,1,2.289196,0.170533,18.243341,33.157086,0.01,20000,0,cpu


In [ ]:
m.save(NameFile.nameit(params),ResDir,file_num) #mooie naam voor de results file met alle parameter values erin

In [7]:
loss

tensor(2.3038, device='cuda:0', grad_fn=<NllLossBackward>)

In [8]:
batch

[tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]]],
 
 
         [[[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]]],
 
 
         [[[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]]],
 
 
         ...,
 
 
         [[[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ..

In [10]:
pd.DataFrame.from_dict(m.run_data,orient='columns').sort_values('epoch duration')


,run,epoch,loss,accuracy,epoch duration,run duration,lr,batch_size,num_workers,device
0,1,1,1.046105,0.598533,6.778939,7.394323,0.01,1000,0,cuda
6,7,1,2.148680,0.180700,7.740357,14.333302,0.01,10000,1,cuda
2,3,1,1.023789,0.602033,9.261346,12.786363,0.01,1000,1,cuda
8,9,1,2.289859,0.127833,10.266546,17.618935,0.01,20000,0,cuda
4,5,1,2.148473,0.185933,11.915739,17.451818,0.01,10000,0,cuda
10,11,1,2.250920,0.201850,13.226789,24.662409,0.01,20000,1,cuda
7,8,1,2.137560,0.199417,14.550092,24.784825,0.01,10000,1,cpu
11,12,1,2.268184,0.221817,16.631756,36.190217,0.01,20000,1,cpu
5,6,1,2.087540,0.271700,17.762079,27.343355,0.01,10000,0,cpu
9,10,1,2.289196,0.170533,18.243341,33.157086,0.01,20000,0,cpu
